<a href="https://colab.research.google.com/github/kaushikabhishek87/Tensorflow_projects/blob/main/11_text_generation_with_RNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers.experimental import preprocessing
import numpy as np
import os
import time
import pandas as pd

In [ ]:
!wget https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt

--2021-07-09 08:04:47--  https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt
Resolving storage.googleapis.com (storage.googleapis.com)... 172.217.13.240, 172.217.15.80, 172.217.9.208, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|172.217.13.240|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1115394 (1.1M) [text/plain]
Saving to: ‘shakespeare.txt’

shakespeare.txt     100%[===================>]   1.06M  --.-KB/s    in 0.01s   

2021-07-09 08:04:47 (107 MB/s) - ‘shakespeare.txt’ saved [1115394/1115394]



In [ ]:
text = open("/content/shakespeare.txt", mode="rb" ).read().decode(encoding="utf8")

In [ ]:
# text = open("/content/shakespeare.txt", mode="r" ).read()

In [ ]:
print(text[:250])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.



In [ ]:
# Unique characters 
vocab = set(text)
print(len(vocab))

65


## Preprocess Text

In [ ]:
example_text = ["there was just an earthquake", "hoping god everyone is safe"]
chars = tf.strings.unicode_split(example_text, input_encoding="UTF-8")
chars

<tf.RaggedTensor [[b't', b'h', b'e', b'r', b'e', b' ', b'w', b'a', b's', b' ', b'j', b'u', b's', b't', b' ', b'a', b'n', b' ', b'e', b'a', b'r', b't', b'h', b'q', b'u', b'a', b'k', b'e'], [b'h', b'o', b'p', b'i', b'n', b'g', b' ', b'g', b'o', b'd', b' ', b'e', b'v', b'e', b'r', b'y', b'o', b'n', b'e', b' ', b'i', b's', b' ', b's', b'a', b'f', b'e']]>

In [ ]:
chars.shape

TensorShape([2, None])

In [ ]:
ids_from_chars = preprocessing.StringLookup(vocabulary=list(vocab) , 
                                            mask_token=None)

In [ ]:
chars[0]

<tf.Tensor: shape=(28,), dtype=string, numpy=
array([b't', b'h', b'e', b'r', b'e', b' ', b'w', b'a', b's', b' ', b'j',
       b'u', b's', b't', b' ', b'a', b'n', b' ', b'e', b'a', b'r', b't',
       b'h', b'q', b'u', b'a', b'k', b'e'], dtype=object)>

In [ ]:
ids = ids_from_chars(chars)
ids[0]

<tf.Tensor: shape=(28,), dtype=int64, numpy=
array([14, 28, 52, 31, 52, 62, 39, 49, 33, 62, 58, 45, 33, 14, 62, 49, 22,
       62, 52, 49, 31, 14, 28, 26, 45, 49, 23, 52])>

In [ ]:
chars_from_ids = preprocessing.StringLookup(vocabulary=ids_from_chars.get_vocabulary(),
                                            invert = True, mask_token=None)
chars = chars_from_ids(ids)
chars

<tf.RaggedTensor [[b't', b'h', b'e', b'r', b'e', b' ', b'w', b'a', b's', b' ', b'j', b'u', b's', b't', b' ', b'a', b'n', b' ', b'e', b'a', b'r', b't', b'h', b'q', b'u', b'a', b'k', b'e'], [b'h', b'o', b'p', b'i', b'n', b'g', b' ', b'g', b'o', b'd', b' ', b'e', b'v', b'e', b'r', b'y', b'o', b'n', b'e', b' ', b'i', b's', b' ', b's', b'a', b'f', b'e']]>

In [ ]:
tf.strings.reduce_join(chars, axis=-1).numpy()

array([b'there was just an earthquake', b'hoping god everyone is safe'],
      dtype=object)

In [ ]:
def text_from_ids(ids):
  return tf.strings.reduce_join(chars_from_ids(ids), axis=-1).numpy()

In [ ]:
text_from_ids(ids)

array([b'there was just an earthquake', b'hoping god everyone is safe'],
      dtype=object)

## Creating Training Exmaples & Target

In [ ]:
all_ids = ids_from_chars(tf.strings.unicode_split(text, input_encoding="UTF-8"))
all_ids

<tf.Tensor: shape=(1115394,), dtype=int64, numpy=array([ 3,  8, 31, ..., 40, 56,  7])>

In [ ]:
ids_dataset = tf.data.Dataset.from_tensor_slices(all_ids)
ids_dataset

<TensorSliceDataset shapes: (), types: tf.int64>

In [ ]:
for i in ids_dataset.take(10):
  print(chars_from_ids(i).numpy().decode("UTF-8") )


F
i
r
s
t
 
C
i
t
i


In [ ]:
seq_length = 100
examples_per_epoch = len(text)//(seq_length+1)
examples_per_epoch

11043

In [ ]:
sequences = ids_dataset.batch(seq_length+1, drop_remainder=True)

for i in sequences.take(1):
  print(chars_from_ids(i))

tf.Tensor(
[b'F' b'i' b'r' b's' b't' b' ' b'C' b'i' b't' b'i' b'z' b'e' b'n' b':'
 b'\n' b'B' b'e' b'f' b'o' b'r' b'e' b' ' b'w' b'e' b' ' b'p' b'r' b'o'
 b'c' b'e' b'e' b'd' b' ' b'a' b'n' b'y' b' ' b'f' b'u' b'r' b't' b'h'
 b'e' b'r' b',' b' ' b'h' b'e' b'a' b'r' b' ' b'm' b'e' b' ' b's' b'p'
 b'e' b'a' b'k' b'.' b'\n' b'\n' b'A' b'l' b'l' b':' b'\n' b'S' b'p' b'e'
 b'a' b'k' b',' b' ' b's' b'p' b'e' b'a' b'k' b'.' b'\n' b'\n' b'F' b'i'
 b'r' b's' b't' b' ' b'C' b'i' b't' b'i' b'z' b'e' b'n' b':' b'\n' b'Y'
 b'o' b'u' b' '], shape=(101,), dtype=string)


In [ ]:
for i in sequences.take(5):
  print(text_from_ids(i))

b'First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou '
b'are all resolved rather to die than to famish?\n\nAll:\nResolved. resolved.\n\nFirst Citizen:\nFirst, you k'
b"now Caius Marcius is chief enemy to the people.\n\nAll:\nWe know't, we know't.\n\nFirst Citizen:\nLet us ki"
b"ll him, and we'll have corn at our own price.\nIs't a verdict?\n\nAll:\nNo more talking on't; let it be d"
b'one: away, away!\n\nSecond Citizen:\nOne word, good citizens.\n\nFirst Citizen:\nWe are accounted poor citi'


In [ ]:
def split_input_target(sequence):
  input_text = sequence[:-1]
  target_text = sequence[1:]
  return input_text, target_text


In [ ]:
split_input_target(list("Tensorflow"))

(['T', 'e', 'n', 's', 'o', 'r', 'f', 'l', 'o'],
 ['e', 'n', 's', 'o', 'r', 'f', 'l', 'o', 'w'])

In [ ]:
dataset = sequences.map(split_input_target)

In [ ]:
for input_example, target_example in dataset.take(1):
  print(text_from_ids(input_example))
  print(text_from_ids(target_example))


b'First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou'
b'irst Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou '


## Creating training batches

In [ ]:
BATCH_SIZE = 64
BUFFER_SIZE = 1000

dataset = (
    dataset
    .shuffle(BUFFER_SIZE)
    .batch(BATCH_SIZE, drop_remainder=True)
    .prefetch(tf.data.AUTOTUNE)
)
dataset

<PrefetchDataset shapes: ((64, 100), (64, 100)), types: (tf.int64, tf.int64)>

In [ ]:
dataset.take(1)

<TakeDataset shapes: ((64, 100), (64, 100)), types: (tf.int64, tf.int64)>

## Model

In [ ]:
# Length of the vocabulary in chars
vocab_size = len(vocab)

# The embedding dimension
embedding_dim = 256

# Number of RNN units
rnn_units = 1024

class MyModel(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim, rnn_units):
    super().__init__(self)
    self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
    self.gru = tf.keras.layers.GRU(rnn_units,
                                   return_sequences=True,
                                   return_state=True)
    self.dense = tf.keras.layers.Dense(vocab_size)

  def call(self, inputs, states=None, return_state=False, training=False):
    x = inputs
    x = self.embedding(x, training=training)
    if states is None:
      states = self.gru.get_initial_state(x)
    x, states = self.gru(x, initial_state=states, training=training)
    x = self.dense(x, training=training)

    if return_state:
      return x, states
    else:
      return x

In [ ]:
model = MyModel(
    # Be sure the vocabulary size matches the `StringLookup` layers.
    vocab_size=len(ids_from_chars.get_vocabulary()),
    embedding_dim=embedding_dim,
    rnn_units=rnn_units)

In [ ]:
for input_example_batch, target_example_batch in dataset.take(1):
    example_batch_predictions = model(input_example_batch)
    print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(64, 100, 66) # (batch_size, sequence_length, vocab_size)


In [ ]:
model.summary()

Model: "my_model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      multiple                  16896     
_________________________________________________________________
gru_1 (GRU)                  multiple                  3938304   
_________________________________________________________________
dense_1 (Dense)              multiple                  67650     
Total params: 4,022,850
Trainable params: 4,022,850
Non-trainable params: 0
_________________________________________________________________


In [ ]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices, axis=-1).numpy()

In [ ]:
sampled_indices

array([58, 31,  7, 42, 44, 29,  9, 45, 39, 41,  2, 58,  6, 28, 12, 23, 36,
       22, 35,  8, 56,  8, 63, 40, 41,  0, 33, 16, 63, 39, 62, 14, 30, 30,
       32, 63, 33, 29, 37, 29, 64, 56, 10,  2, 43, 51, 10, 48,  9, 39, 22,
       61, 22, 64, 57, 16, 61, 35,  0, 38, 32, 53, 37, 19, 22, 59, 11, 14,
       52, 34, 40, 34, 28, 32, 25,  9, 57, 43, 55,  7,  9, 51, 20, 54, 39,
       27, 28, 48, 44, 47, 48, 35, 55,  1, 42, 34,  1, 29, 39, 60])

In [ ]:
print("Input:\n", text_from_ids(input_example_batch[0]).numpy())
print()
print("Next Char Predictions:\n", text_from_ids(sampled_indices).numpy())

AttributeError: ignored

In [ ]:
loss = tf.losses.SparseCategoricalCrossentropy(from_logits=True)

In [ ]:
example_batch_loss = loss(target_example_batch, example_batch_predictions)
mean_loss = example_batch_loss.numpy().mean()
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print("Mean loss:        ", mean_loss)

Prediction shape:  (64, 100, 66)  # (batch_size, sequence_length, vocab_size)
Mean loss:         4.189619


In [ ]:
tf.exp(mean_loss).numpy()

65.99765

In [ ]:
model.compile(optimizer='adam', loss=loss)

In [ ]:
EPOCHS = 20
history = model.fit(dataset, epochs=EPOCHS)

Epoch 1/20
172/172 [==============================] - 12s 55ms/step - loss: 2.6684
Epoch 2/20
172/172 [==============================] - 10s 55ms/step - loss: 1.9777
Epoch 3/20
172/172 [==============================] - 10s 56ms/step - loss: 1.7175
Epoch 4/20
172/172 [==============================] - 10s 57ms/step - loss: 1.5508
Epoch 5/20
172/172 [==============================] - 10s 58ms/step - loss: 1.4490
Epoch 6/20
172/172 [==============================] - 10s 59ms/step - loss: 1.3792
Epoch 7/20
172/172 [==============================] - 10s 58ms/step - loss: 1.3253
Epoch 8/20
172/172 [==============================] - 10s 57ms/step - loss: 1.2803
Epoch 9/20
172/172 [==============================] - 10s 57ms/step - loss: 1.2393
Epoch 10/20
172/172 [==============================] - 10s 57ms/step - loss: 1.1989
Epoch 11/20
172/172 [==============================] - 10s 57ms/step - loss: 1.1572
Epoch 12/20
172/172 [==============================] - 10s 57ms/step - loss: 1.1161
E

In [ ]:
class OneStep(tf.keras.Model):
  def __init__(self, model, chars_from_ids, ids_from_chars, temperature=1.0):
    super().__init__()
    self.temperature = temperature
    self.model = model
    self.chars_from_ids = chars_from_ids
    self.ids_from_chars = ids_from_chars

    # Create a mask to prevent "[UNK]" from being generated.
    skip_ids = self.ids_from_chars(['[UNK]'])[:, None]
    sparse_mask = tf.SparseTensor(
        # Put a -inf at each bad index.
        values=[-float('inf')]*len(skip_ids),
        indices=skip_ids,
        # Match the shape to the vocabulary
        dense_shape=[len(ids_from_chars.get_vocabulary())])
    self.prediction_mask = tf.sparse.to_dense(sparse_mask)

  @tf.function
  def generate_one_step(self, inputs, states=None):
    # Convert strings to token IDs.
    input_chars = tf.strings.unicode_split(inputs, 'UTF-8')
    input_ids = self.ids_from_chars(input_chars).to_tensor()

    # Run the model.
    # predicted_logits.shape is [batch, char, next_char_logits]
    predicted_logits, states = self.model(inputs=input_ids, states=states,
                                          return_state=True)
    # Only use the last prediction.
    predicted_logits = predicted_logits[:, -1, :]
    predicted_logits = predicted_logits/self.temperature
    # Apply the prediction mask: prevent "[UNK]" from being generated.
    predicted_logits = predicted_logits + self.prediction_mask

    # Sample the output logits to generate token IDs.
    predicted_ids = tf.random.categorical(predicted_logits, num_samples=1)
    predicted_ids = tf.squeeze(predicted_ids, axis=-1)

    # Convert from token ids to characters
    predicted_chars = self.chars_from_ids(predicted_ids)

    # Return the characters and model state.
    return predicted_chars, states

In [ ]:
one_step_model = OneStep(model, chars_from_ids, ids_from_chars)

In [ ]:
start = time.time()
states = None
next_char = tf.constant(['ROMEO:'])
result = [next_char]

for n in range(1000):
  next_char, states = one_step_model.generate_one_step(next_char, states=states)
  result.append(next_char)

result = tf.strings.join(result)
end = time.time()
print(result[0].numpy().decode('utf-8'), '\n\n' + '_'*80)
print('\nRun time:', end - start)

TypeError: ignored

## Keras API Model

In [ ]:
dataset.take(1)

<TakeDataset shapes: ((64, 100), (64, 100)), types: (tf.int64, tf.int64)>

In [ ]:
embedding = tf.keras.layers.Embedding(input_dim=len(ids_from_chars.get_vocabulary()), output_dim= embedding_dim)

inputs = tf.keras.layers.Input(shape = (100,), dtype = tf.int64 )
X = embedding(inputs)
X = tf.keras.layers.GRU(units=rnn_units, return_sequences=True)(X)
# X = tf.keras.layers.Flatten()(X)
outputs = tf.keras.layers.Dense(units=len(ids_from_chars.get_vocabulary()))(X)

model_seq = tf.keras.models.Model(inputs, outputs)

model_seq.summary()


Model: "model_21"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_26 (InputLayer)        [(None, 100)]             0         
_________________________________________________________________
embedding_26 (Embedding)     (None, 100, 256)          16896     
_________________________________________________________________
gru_26 (GRU)                 (None, 100, 1024)         3938304   
_________________________________________________________________
dense_25 (Dense)             (None, 100, 66)           67650     
Total params: 4,022,850
Trainable params: 4,022,850
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model_seq.compile(loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                  optimizer="Adam",
                  metrics=["accuracy"])

In [ ]:
history_model_seq = model_seq.fit(dataset, epochs=20)

Epoch 1/20
172/172 [==============================] - 12s 59ms/step - loss: 2.7349 - accuracy: 0.2750
Epoch 2/20
172/172 [==============================] - 10s 59ms/step - loss: 2.0124 - accuracy: 0.4135
Epoch 3/20
172/172 [==============================] - 10s 57ms/step - loss: 1.7476 - accuracy: 0.4837
Epoch 4/20
172/172 [==============================] - 10s 57ms/step - loss: 1.5799 - accuracy: 0.5281
Epoch 5/20
172/172 [==============================] - 10s 57ms/step - loss: 1.4712 - accuracy: 0.5560
Epoch 6/20
172/172 [==============================] - 10s 57ms/step - loss: 1.3977 - accuracy: 0.5746
Epoch 7/20
172/172 [==============================] - 10s 57ms/step - loss: 1.3425 - accuracy: 0.5882
Epoch 8/20
172/172 [==============================] - 10s 57ms/step - loss: 1.2961 - accuracy: 0.6000
Epoch 9/20
172/172 [==============================] - 10s 57ms/step - loss: 1.2528 - accuracy: 0.6122
Epoch 10/20
172/172 [==============================] - 10s 57ms/step - loss: 1.212

In [ ]:
model_seq.save("model_seq.h5")

In [ ]:
model_loaded = tf.keras.models.load_model("model_seq.h5")

In [ ]:
next_char = ["Romeo:"]

In [ ]:
model_seq( )

In [ ]:
start = time.time()
states = None
next_char = tf.constant(['ROMEO:'])
result = [next_char]

for n in range(1000):
  next_char, states = one_step_model.generate_one_step(next_char, states=states)
  result.append(next_char)

result = tf.strings.join(result)
end = time.time()
print(result[0].numpy().decode('utf-8'), '\n\n' + '_'*80)
print('\nRun time:', end - start)

TypeError: ignored

AttributeError: ignored